In [10]:
import pandas as pd
from x_x import account_info as ai
from pyjin import pyjin

In [11]:
json_input = [{'pid': 12, 'net_contents_prob': '0.0', 'reordered_PT_cnt': 29},
 {'pid': 13,'net_contents_prob': '0.0', 'reordered_PT_cnt': 23},
 {'pid': 14,'net_contents_prob': '0.0', 'reordered_PT_cnt': 43},
 {'pid': 15,'net_contents_prob': '0.01', 'reordered_PT_cnt': 1},
 {'pid': 16,'net_contents_prob': '0.0', 'reordered_PT_cnt': 9},
 {'pid': 17,'net_contents_prob': '0.0', 'reordered_PT_cnt': 4}]

In [46]:
json_input

[{'pid': 12, 'net_contents_prob': '0.0', 'reordered_PT_cnt': 29},
 {'pid': 13, 'net_contents_prob': '0.0', 'reordered_PT_cnt': 23},
 {'pid': 14, 'net_contents_prob': '0.0', 'reordered_PT_cnt': 43},
 {'pid': 15, 'net_contents_prob': '0.01', 'reordered_PT_cnt': 1},
 {'pid': 16, 'net_contents_prob': '0.0', 'reordered_PT_cnt': 9},
 {'pid': 17, 'net_contents_prob': '0.0', 'reordered_PT_cnt': 4}]

In [51]:
columns = pd.DataFrame(json_input).columns

In [56]:
key_columns = columns[0]
set_columns = columns[1:]



In [62]:
pd.DataFrame(json_input)[set_columns].to_json(orient='records')

'[{"net_contents_prob":"0.0","reordered_PT_cnt":29},{"net_contents_prob":"0.0","reordered_PT_cnt":23},{"net_contents_prob":"0.0","reordered_PT_cnt":43},{"net_contents_prob":"0.01","reordered_PT_cnt":1},{"net_contents_prob":"0.0","reordered_PT_cnt":9},{"net_contents_prob":"0.0","reordered_PT_cnt":4}]'

In [57]:
set_columns

Index(['net_contents_prob', 'reordered_PT_cnt'], dtype='object')

In [15]:
raw = [{'pid': 12},
 {'pid': 13},
 {'pid': 14},
 {'pid': 15},
 {'pid': 16},
 {'pid': 17}]

In [12]:
acc = ai.vmdb_sql_info(db='yunsik_test')

In [13]:
def bulk_insert_table(insert_json_list, table_name, to_conn):
    # dummy table 생성
    pd.DataFrame(insert_json_list).to_sql(table_name, index=False, if_exists='replace', con=to_conn ,method='multi', chunksize=300)
    


In [14]:
with pyjin.connectDB(**acc) as con:
    bulk_insert_table(json_input, 'test_dummy', con)

In [16]:
with pyjin.connectDB(**acc) as con:
    bulk_insert_table(raw, 'test', con)

In [64]:
list_dict_condition = [{'pid': 12},
 {'pid': 13},
 {'pid': 14},
 {'pid': 15},
 {'pid': 16},
 {'pid': 17}]

In [65]:
list_dict_input = [{'net_contents_prob': '0.0', 'reordered_PT_cnt': 29},
 {'net_contents_prob': '0.0', 'reordered_PT_cnt': 23},
 {'net_contents_prob': '0.0', 'reordered_PT_cnt': 43},
 {'net_contents_prob': '0.01', 'reordered_PT_cnt': 1},
 {'net_contents_prob': '0.0', 'reordered_PT_cnt': 9},
 {'net_contents_prob': '0.0', 'reordered_PT_cnt': 4}]

In [84]:
bulk_join_update(acc=ai.vmdb_sql_info(),
                 db='yunsik_test',
                 table='test',
                 list_dict_input=list_dict_input,
                 list_dict_condition=list_dict_condition)

   pid net_contents_prob  reordered_PT_cnt
0   12               0.0                29
1   13               0.0                23
2   14               0.0                43
3   15              0.01                 1
4   16               0.0                 9
5   17               0.0                 4
dummy table 생성완료
query error :  (pymysql.err.ProgrammingError) (1146, "Table 'yunsik_test.test_dummy' doesn't exist")
[SQL: 
            update yunsik_test.test A
            inner join yunsik_test.test_dummy B 
            on A.pid = B.pid
            set A.net_contents_prob = B.net_contents_prob,A.reordered_PT_cnt = B.reordered_PT_cnt
            ]
(Background on this error at: http://sqlalche.me/e/14/f405)
test update failed, (pymysql.err.ProgrammingError) (1146, "Table 'yunsik_test.test_dummy' doesn't exist")
[SQL: 
            update yunsik_test.test A
            inner join yunsik_test.test_dummy B 
            on A.pid = B.pid
            set A.net_contents_prob = B.net_contents_prob

In [89]:
def bulk_join_update(acc,
                     db,
                     table,
                     list_dict_input,
                     list_dict_condition):

    set_conditions = ",".join(["A.{key} = B.{key}".format(key=key) for key, value in list_dict_input[0].items()])
    join_conditions = ",".join(["A.{key} = B.{key}".format(key=key) for key, value in list_dict_condition[0].items()])
    
	# dummy table
    df = pd.concat([pd.DataFrame(list_dict_condition),pd.DataFrame(list_dict_input)],axis=1)
    print(df)
    join_query = """
            update {db}.{table} A
            inner join {db}.{table_dummy} B 
            on {join_conditions}
            set {set_conditions}
            """.format(db=db,
                       table=table,
                       table_dummy=table+'_dummy',
                       join_conditions=join_conditions,
                       set_conditions=set_conditions)

    with pyjin.connectDB(**acc) as con:
        try:
            # dummy table이 없으면 새로 create하고, 있으면 drop create insert. (if_exists='replace')
            df.to_sql(table+'_dummy', schema=db, con=con, if_exists='replace', index=False, chunksize=300, method='multi')
            pyjin.print_logging("dummy table 업뎃 완료")
            ##  join update
            pyjin.execute_query(con,join_query)

        except Exception as e:
            pyjin.print_logging("{} update failed, {}".format(table, e))

    pyjin.print_logging('update completed')


In [90]:
bulk_join_update(acc=ai.vmdb_sql_info(),
                 db='yunsik_test',
                 table='test',
                 list_dict_input=list_dict_input,
                 list_dict_condition=list_dict_condition)

   pid net_contents_prob  reordered_PT_cnt
0   12               0.0                29
1   13               0.0                23
2   14               0.0                43
3   15              0.01                 1
4   16               0.0                 9
5   17               0.0                 4
dummy table 업뎃 완료
update completed
